In [1]:
using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
using Revise
using EasyHybrid
using Lux
using Optimisers
using WGLMakie
using Random
using LuxCore
using CSV, DataFrames
using EasyHybrid.MLUtils
using Statistics
using Plots
using JLD2

In [2]:
# 02 - multivariate NN
testid = "02_multiNN";
version = "v20251125";
results_dir = joinpath(@__DIR__, "eval");
targets = [:BD, :SOCconc, :SOCdensity, :CF];

# input
df = CSV.read(joinpath(@__DIR__, "data/lucas_preprocessed_$version.csv"), DataFrame; normalizenames=true)

# scales
scalers = Dict(
    :SOCconc   => 0.151, # g/kg, log(x+1)*0.151
    :CF        => 0.263, # percent, log(x+1)*0.263
    :BD        => 0.529, # g/cm3, x*0.529
    :SOCdensity => 0.167, # kg/m3, log(x)*0.167
);

# predictor
predictors = Symbol.(names(df))[18:end-6] # CHECK EVERY TIME 
nf = length(predictors)

# configuration
# search space
hidden_configs = [ 
    (512, 256, 128, 64, 32, 16),
    (512, 256, 128, 64, 32), 
    (256, 128, 64, 32, 16),
    (256, 128, 64, 32),
    (256, 128, 64),
    (128, 64, 32, 16),
    (128, 64, 32),
    (64, 32, 16)
];
batch_sizes = [128, 256, 512];
lrs = [1e-3, 5e-4, 1e-4];
activations = [relu, swish, gelu];

configs = [(h=h, bs=bs, lr=lr, act=act)
           for h in hidden_configs
           for bs in batch_sizes
           for lr in lrs
           for act in activations]

println(length(configs))


216


In [3]:
# cross-validation
k = 5;
folds = make_folds(df, k = k, shuffle = true);
rlt_list_param = Vector{DataFrame}(undef, k);
rlt_list_pred = Vector{DataFrame}(undef, k);  
@info "Threads available: $(Threads.nthreads())"


┌ Info: Threads available: 1
└ @ Main /home/xuemeng/work_xuemeng/EasyDensity.jl/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:6


In [ ]:
@time for test_fold in 1:k
    @info "Training outer fold $test_fold of $k"

    train_folds = setdiff(1:k, test_fold)
    train_idx = findall(in(train_folds), folds)
    train_df = df[train_idx, :]
    test_idx  = findall(==(test_fold), folds)
    test_df = df[test_idx, :]
    
    # track best config for this outer fold
    lk = ReentrantLock()
    best_val_loss = Inf
    best_config = nothing
    best_result = nothing
    best_model = nothing
    best_model_path = nothing
    results_param = DataFrame(h=String[], bs=Int[], lr=Float64[], act=String[], r2=Float64[], mse=Float64[], best_epoch=Int[], test_fold=Int[])

    # param search by looping....    
    Threads.@threads for i in 1:length(configs)
        try
            cfg = configs[i]
        
            h  = cfg.h
            bs = cfg.bs
            lr = cfg.lr
            act = cfg.act
            println("Testing h=$h, bs=$bs, lr=$lr, activation=$act")
        
            nn_local = EasyHybrid.constructNNModel(
                predictors, targets;
                hidden_layers = collect(h),
                activation = act,
                scale_nn_outputs = true,
                input_batchnorm = false
            )
            
            rlt = train(
                nn_local, train_df, ();
                nepochs = 200,
                batchsize = bs,
                opt = AdamW(lr),
                training_loss = :mse,
                loss_types = [:mse, :r2],
                shuffleobs = true,
                file_name = "$(testid)_config$(i)_fold$(test_fold).jld2",
                random_seed = 42,
                patience = 15,
                yscale = identity,
                agg = mean,
                return_model = :best,
                show_progress = false,
                plotting = false,
                hybrid_name = "$(testid)_config$(i)_fold$(test_fold)"
            )
    
            lock(lk)
            if rlt.best_loss < best_val_loss
                best_val_loss = rlt.best_loss
                best_config = cfg
                best_result = rlt
                best_model = deepcopy(nn_local)
                best_model_path = "best_model_$(testid)_config$(i)_fold$(test_fold)"
            end
            unlock(lk)
        catch err
            @error "Thread $i crashed" exception = err
            @error sprint(showerror, err)
        end
    end

    # register best hyper paramets
    agg_name = Symbol("mean")
    r2s  = map(vh -> getproperty(vh, agg_name), best_result.val_history.r2)
    mses = map(vh -> getproperty(vh, agg_name), best_result.val_history.mse)
    best_epoch = max(best_result.best_epoch, 1)

    local_results_param = DataFrame(
        h = string(best_config.h),
        bs = best_config.bs,
        lr = best_config.lr,
        act = string(best_config.act),
        r2 = r2s[best_epoch],
        mse = mses[best_epoch],
        best_epoch = best_epoch,
        test_fold = test_fold,
        path = best_model_path,
    )
    rlt_list_param[test_fold] = local_results_param

    # remove tmp files
    cp(joinpath("output_tmp", best_model_path * ".jld2"), joinpath("models", best_model_path * ".jld2"); force=true) 
    for f in readdir("output_tmp"; join=true)
        rm(f; force=true, recursive=true)
    end
    

    (x_test,  y_test)  = prepare_data(best_model, test_df)
    ps, st = best_result.ps, best_result.st
    ŷ_test, st_test = best_model(x_test, ps, LuxCore.testmode(st))
    # println(propertynames(ŷ_test))

    for var in [:BD, :SOCconc, :CF, :SOCdensity]
        if hasproperty(ŷ_test, var)
            val = getproperty(ŷ_test, var)

            if val isa AbstractVector && length(val) == nrow(test_df)
                test_df[!, Symbol("pred_", var)] = val # per row

            elseif (val isa Number) || (val isa AbstractVector && length(val) == 1)
                test_df[!, Symbol("pred_", var)] = fill(Float32(val isa AbstractVector ? first(val) : val), nrow(test_df))
            end


        end
    end
    
    rlt_list_pred[test_fold] = test_df
end

rlt_param = vcat(rlt_list_param...)
rlt_pred = vcat(rlt_list_pred...)

CSV.write(joinpath(results_dir, "$(testid)_cv.pred_$version.csv"), rlt_pred)
CSV.write(joinpath(results_dir, "$(testid)_hyperparams_$version.csv"), rlt_param)



[ Info: Training outer fold 1 of 5


Testing h=(512, 256, 128, 64, 32, 16), bs=128, lr=0.001, activation=relu


[ Info: Plotting disabled.
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 20 with best validation loss wrt mse: 0.017371590157513557
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 5 of 200 epochs with best validation loss wrt mse: 0.017371590157513557
[ Info: Training outer fold 2 of 5
[ Info: Plotting disabled.


Testing h=(512, 256, 128, 64, 32, 16), bs=128, lr=0.001, activation=relu


[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 21 with best validation loss wrt mse: 0.018741458112512516
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 6 of 200 epochs with best validation loss wrt mse: 0.018741458112512516
[ Info: Training outer fold 3 of 5
[ Info: Plotting disabled.


...

[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 49 with best validation loss wrt mse: 0.017500435378340687
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 34 of 200 epochs with best validation loss wrt mse: 0.017500435378340687
[ Info: Training outer fold 5 of 5


Testing h=(512, 256, 128, 64, 32, 16), bs=128, lr=0.001, activation=relu


[ Info: Plotting disabled.
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 27 with best validation loss wrt mse: 0.017496516447373094
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 12 of 200 epochs with best validation loss wrt mse: 0.017496516447373094


945.751545 seconds (2.32 G allocations: 306.092 GiB, 8.91% gc time, 72.56% compilation time: <1% of which was recompilation)


"/mnt/tupi/HybridModeling/EasyDensity.jl/eval/02_multiNN_hyperparams_v20251125.csv"

In [12]:
rlt_param

Row,h,bs,lr,act,r2,mse,best_epoch,test_fold,path
,String,Int64,Float64,String,Float64,Float64,Int64,Int64,String
1,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.22056,0.0185874,5,1,best_model_02_multiNN_config1_fold1
2,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.235852,0.0203727,6,2,best_model_02_multiNN_config1_fold2
3,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.279135,0.019365,17,3,best_model_02_multiNN_config1_fold3
4,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.276259,0.0178793,34,4,best_model_02_multiNN_config1_fold4
5,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.289418,0.0176353,12,5,best_model_02_multiNN_config1_fold5


In [5]:
bad_rows = rlt_pred[rlt_pred.SOCdensity .< 0, :]


Row,h,bs,lr,act,r2,mse,best_epoch,test_fold,path
,String,Int64,Float64,String,Float64,Float64,Int64,Int64,String
1,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.22056,0.0185874,5,1,best_model_02_multiNN_config1_fold1
2,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.235852,0.0203727,6,2,best_model_02_multiNN_config1_fold2
3,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.279135,0.019365,17,3,best_model_02_multiNN_config1_fold3
4,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.276259,0.0178793,34,4,best_model_02_multiNN_config1_fold4
5,"(512, 256, 128, 64, 32, 16)",128,0.001,relu,0.289418,0.0176353,12,5,best_model_02_multiNN_config1_fold5


In [11]:
println(size(rlt_pred))

(56117, 389)
